In [1]:
import wandb
import numpy as np
import torch
import cv2 as cv
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import RMSprop
from torch.optim.lr_scheduler import StepLR
from data_loaders import CAERSDataset
from utils import get_transform, accuracy, accuracy_julia
from model import model_generator
from tqdm import tqdm
import os.path as osp
from keras.models import load_model
import os

c:\Users\jdts\anaconda3\envs\emotionram\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '.\models\\faces_19_95.01295733262594.pth'
model = torch.load(model_path)


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## Se a maquina não tiver GPU o teste será rodado na CPU

test_dataset = CAERSDataset('data/CAER-S', 'data/test.txt', transforms=get_transform(train=False))

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model.to(device)
loss = CrossEntropyLoss()
opt = RMSprop(model.parameters(), lr=3e-3)
lr_scheduler = StepLR(opt, 60, 0.4)

best_model_acc = 0.0
lowest_loss = 1
count_ep = 0

In [4]:
model.eval() ## modo teste e validação
running_loss = 0.0
test_outs, test_gts = [],[] ##malu

with torch.no_grad():
    with tqdm(total=len(test_dataloader), position=1, leave=True) as pbar_1:
        for data, label in iter(test_dataloader):
            face = data['face']

            preds = model(face.to(device))
            preds = preds.to('cpu')

            running_loss = loss(preds, label)
            test_outs.extend(torch.argmax(preds, dim=1))##malu
            test_gts.extend(label.to('cpu'))##malu
            pbar_1.update(1)
            #image =  cv.imread(face)
            #cv.imshow('teste',image)
            #cv.waitkey(0)
        
            
                   

    test_acc = accuracy_julia(test_outs,test_gts)
    #print(f'test :: acc {test_acc} :: loss {running_loss}')

100%|██████████| 1532/1532 [06:51<00:00,  3.73it/s]


In [5]:
import os
from urllib import request

def salvar_imagem(imagem, nome_arquivo, pasta):
    # Verifica se a pasta existe
    if not os.path.exists(pasta):
        # Cria a pasta se não existir
        os.makedirs(pasta)

    # Define o caminho completo para o arquivo
    caminho_arquivo = os.path.join(pasta, nome_arquivo)

    # Faz o download da imagem
    #request.urlretrieve(url, caminho_arquivo)
    cv.imwrite(caminho_arquivo,imagem)

    print(f'A imagem {nome_arquivo} foi salva em {pasta}.')

In [6]:


def set_emotion(label):

    if label == '0' :
        emotion = 'Angry'
    elif label == '1':
        emotion = 'Disgust'
    elif label == '2':
        emotion = 'Fear'
    elif label == '3':
        emotion = 'Happy'
    elif label == '4':
        emotion = 'Neutral'
    elif label == '5':
        emotion = 'Sad'
    elif label == '6':
        emotion = 'Surprise'
    return emotion
    
        
              


In [7]:
data_quali = open("data\\test.txt")
print("ok1")
file_content = data_quali.readlines()
i=0
#dado = []
num = ''
emotion = ''

for i in range(len(file_content)):
    dado = file_content[i].split(',')
    path_image = "data\CAER-S\\" + dado[0]
    ground_truth = dado[1]

    image = cv.imread(path_image)
    aux = dado[0].split('/')
    preds = str(test_outs[i].numpy())


    emotion_GT = str(set_emotion(ground_truth))
    emption_Pred =str(set_emotion(preds))
    name_image = str(aux[2]) + emotion_GT + emption_Pred + '.png'
    
    salvar_imagem(image, name_image,"imagens_quali")
    #cv.imshow(ground_truth,image)
    #cv.waitKey(0)
    
    #print(emotion_GT, emption_Pred)

ok1
A imagem 0001.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0002.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0003.pngAngryHappy.png foi salva em imagens_quali.
A imagem 0004.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0005.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0006.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0007.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0008.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0009.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0010.pngAngrySurprise.png foi salva em imagens_quali.
A imagem 0011.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0012.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0013.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0014.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0015.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0016.pngAngryAngry.png foi salva em imagens_quali.
A imagem 0017.pngAngryDisgust.png

KeyboardInterrupt: 